In [1]:
!pip install keras

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1125)'))) - skipping


In [2]:
from keras.utils.data_utils import get_file
path='mnist.npz'
path = get_file(path,origin='https://s3.amazonaws.com/img-datasets/mnist.npz',file_hash='8a61469f7ea1b51cbae51d4f78837e45')
print(path)

#打开Mnist数据
def load_data():
    path='mnist2.pkl.gz'
    path = get_file(path,origin='https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz')
    print(path)
    f = gzip.open(path, 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")

C:\Users\86189\.keras\datasets\mnist.npz


In [3]:
"""
network.py
~~~~~~~~~~
IT WORKS

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
jsxyhelu添加了适当的中文注释
"""

#### Libraries
# Standard library
import random
# Third-party libraries
import numpy as np

class GoNetwork(object):

    def __init__(self, sizes):
        """size代表的是网络的分层结构，比如[2, 3, 1]
        The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes) #层数
        self.sizes = sizes #每层size
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]   #随机生成子节点
        # net.weights[1] 是⼀个存储着连接第⼆层和第三层神经元权重的 Numpy 矩阵。
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
    #前向网络,主要用于测试当前网络                      
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    #随机梯度下降算法
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        #⾸先随机地将训练数据打乱
        for j in range(epochs):
            random.shuffle(training_data)
            #再将它分成多个适当⼤⼩的⼩批量数据
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            #最主要的一行代码

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:

                print(self.evaluate(test_data))
            else:
                print("Epoch {} complete".format(j))

    #根据单次梯度下降的迭代更新⽹络的权重和偏置
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    #反向传播就是一种快速计算代价函数梯度的方法，也就是计算delta的一种方法
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1]) #bp1
        nabla_b[-1] = delta  #bp3
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())  #bp4
        
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp #bp2,注意这里的+1，其实是计算了下一层了
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
        
    #evaluate评价函数
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)))
                        for (x, y) in test_data]
        return test_results
    
    #cost代价函数
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#########helper函数########
#计算sigmoid，这个函数来自定义
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

#计算sigmoid的导数，这个函数可以被证明
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [56]:
'''
GoDateSets:
GreenOpen 系列
引入数据集
by:jsxyhelu 2018/3/31
'''
import cv2
import os
import pickle
import gzip
import numpy as np
from keras.utils.data_utils import get_file

#打开Mnist数据
def load_data():
    path='mnist2.pkl.gz'
    path = get_file(path,origin='https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz')
    print(path)
    f = gzip.open(path, 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)
def read_directory(directory_name,a = 20,b=0):
    u = len(os.listdir(directory_name))
    k = 0
    pi = np.zeros((1,u),dtype='uint')
    if a == 20:
        for filename in os.listdir(directory_name):
            img = cv2.imread(directory_name + "/" + filename)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            ha = np.zeros((u,1024),dtype='uint')

            for x in range(32):
                for y in range(32):
                    if gray[2*y, 2*x] > 0:
                        ha[k][x*32 + y] = 1
            pi[0][k] = a
            k = k + 1
        return (ha,pi)
    for filename in os.listdir(directory_name):
        
        
        img = cv2.imread(directory_name + "/" + filename)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        ha = np.zeros((u,1024),dtype='uint')
        
        for x in range(32):
            for y in range(32):
                if gray[2*y, 2*x] > 0:
                    ha[k][x*32 + y] = 1
        pi[0][k] = a
        
        
        k = k + 1
    if a > 0:
        ha = np.vstack((b[0], ha))

        pi = np.append(b[1],pi)
    return (ha,pi)
    #return (ha,pi)

#对读取的数据进行重新封装
def load_data_wrapper(a,b):
    tr_d= a
    te_d= b
    training_inputs = [np.reshape(x, (1024, 1)) for x in tr_d[0]]
    
    training_results = [vectorized_result(y) for y in tr_d[1]]
    
    training_data = zip(training_inputs, training_results)
    test_inputs = [np.reshape(x, (1024, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, test_data)
def load_data_wrapper1():
    tr_d, va_d, te_d = load_data()

    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result1(y) for y in tr_d[1]]
#     print(training_inputs)
#     print(training_results)
    aa = list(zip(training_inputs[0:3],training_results[0:3]))
    random.shuffle(aa)
    print(aa)
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

#返回OneHot图
def vectorized_result(j):
    e = np.zeros((15, 1))
    e[j] = 1.0
    return e
def vectorized_result1(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [57]:
import numpy as np
import random

training_data, validation_data, test_data = load_data_wrapper1()

# net = GoNetwork([784, 30, 10])
# net.SGD(training_data, 30, 10, 3.0, test_data=test_data)
# random.shuffle(training_data)

print("asdf")

C:\Users\86189\.keras\datasets\mnist2.pkl.gz
[(array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [

In [6]:
# import numpy as np
# import random
# #training_data, validation_data, test_data = load_data()
# a = read_directory(r"C:\Users\86189\Desktop\py\train\0",0)
# a =read_directory(r"C:\Users\86189\Desktop\py\train\1",1,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\2",2,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\3",3,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\4",4,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\5",5,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\6",6,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\7",7,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\8",8,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\9",9,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\10",10,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\100",11,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\1000",12,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\10000",13,a)
# a= read_directory(r"C:\Users\86189\Desktop\py\train\100000000",14,a)
# b= read_directory(r"C:\Users\86189\Desktop\py\test",20)
# training_data, test_data = load_data_wrapper(a,b)


In [7]:
# import numpy as np
# import random
# training_data, validation_data, test_data = load_data()
# print("sdf",test_data[1])

In [31]:
ii =[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]
po = [[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.01171875]
 [0.0703125 ]
 [0.0703125 ]
 [0.0703125 ]
 [0.4921875 ]
 [0.53125   ]
 [0.68359375]
 [0.1015625 ]
 [0.6484375 ]
 [0.99609375]
 [0.96484375]
 [0.49609375]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.1171875 ]
 [0.140625  ]
 [0.3671875 ]
 [0.6015625 ]
 [0.6640625 ]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.87890625]
 [0.671875  ]
 [0.98828125]
 [0.9453125 ]
 [0.76171875]
 [0.25      ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.19140625]
 [0.9296875 ]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98046875]
 [0.36328125]
 [0.3203125 ]
 [0.3203125 ]
 [0.21875   ]
 [0.15234375]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.0703125 ]
 [0.85546875]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.7734375 ]
 [0.7109375 ]
 [0.96484375]
 [0.94140625]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.3125    ]
 [0.609375  ]
 [0.41796875]
 [0.98828125]
 [0.98828125]
 [0.80078125]
 [0.04296875]
 [0.        ]
 [0.16796875]
 [0.6015625 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.0546875 ]
 [0.00390625]
 [0.6015625 ]
 [0.98828125]
 [0.3515625 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.54296875]
 [0.98828125]
 [0.7421875 ]
 [0.0078125 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.04296875]
 [0.7421875 ]
 [0.98828125]
 [0.2734375 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.13671875]
 [0.94140625]
 [0.87890625]
 [0.625     ]
 [0.421875  ]
 [0.00390625]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.31640625]
 [0.9375    ]
 [0.98828125]
 [0.98828125]
 [0.46484375]
 [0.09765625]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.17578125]
 [0.7265625 ]
 [0.98828125]
 [0.98828125]
 [0.5859375 ]
 [0.10546875]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.0625    ]
 [0.36328125]
 [0.984375  ]
 [0.98828125]
 [0.73046875]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.97265625]
 [0.98828125]
 [0.97265625]
 [0.25      ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.1796875 ]
 [0.5078125 ]
 [0.71484375]
 [0.98828125]
 [0.98828125]
 [0.80859375]
 [0.0078125 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.15234375]
 [0.578125  ]
 [0.89453125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.9765625 ]
 [0.7109375 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.09375   ]
 [0.4453125 ]
 [0.86328125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.78515625]
 [0.3046875 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.08984375]
 [0.2578125 ]
 [0.83203125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.7734375 ]
 [0.31640625]
 [0.0078125 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.0703125 ]
 [0.66796875]
 [0.85546875]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.76171875]
 [0.3125    ]
 [0.03515625]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.21484375]
 [0.671875  ]
 [0.8828125 ]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.953125  ]
 [0.51953125]
 [0.04296875]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.53125   ]
 [0.98828125]
 [0.98828125]
 [0.98828125]
 [0.828125  ]
 [0.52734375]
 [0.515625  ]
 [0.0625    ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]
training_data = zip(pp, ii)
print(training_data)

<>:1: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
<>:11: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
<>:11: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
<ipython-input-31-3b0ee0ee505d>:1: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
  ii =[[0.]


TypeError: list indices must be integers or slices, not float

In [37]:
iii = [[1],[2],[5]]
jjj = [[3],[4]]
print(list(zip(iii,jjj)))

[([1], [3]), ([2], [4])]
